In [1]:
!git clone https://github.com/CS726-AdvML/long_range_TSF.git

Cloning into 'long_range_TSF'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 340 (delta 69), reused 325 (delta 56), pack-reused 0
Receiving objects: 100% (340/340), 19.63 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [1]:
cd /content/long_range_TSF/TFT_NBEATS

/content/long_range_TSF/TFT_NBEATS


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!cp -R /content/gdrive/Shareddrives/CS726-Advanced-Machine-Learning/tft_outputs /content/long_range_TSF

In [4]:
!cp /content/gdrive/Shareddrives/CS726-Advanced-Machine-Learning/favorita-grocery-sales-forecasting.zip /content/long_range_TSF/tft_outputs/data/

In [3]:
# !set -e

# !pip3 install virtualenv # Assumes pip3 is installed!
# !python3 -m virtualenv $OUTPUT_FOLDER/venv
# !source $OUTPUT_FOLDER/venv/bin/activate
# !pip3 install --pre "tensorflow==1.15.*"
!pip3 install -r requirements.txt

!python3 -m script_download_data electricity /content/long_range_TSF/tft_outputs/

#### Running download script ###
Resetting data folder...
Getting electricity data...
Pulling data from https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip to /content/long_range_TSF/tft_outputs/data/electricity/LD2011_2014.txt.zip
done
Unzipping file: /content/long_range_TSF/tft_outputs/data/electricity/LD2011_2014.txt.zip
Done.
Aggregating to hourly data
Processing MT_001
Processing MT_002
Processing MT_003
Processing MT_004
Processing MT_005
Processing MT_006
Processing MT_007
Processing MT_008
Processing MT_009
Processing MT_010
Processing MT_011
Processing MT_012
Processing MT_013
Processing MT_014
Processing MT_015
Processing MT_016
Processing MT_017
Processing MT_018
Processing MT_019
Processing MT_020
Processing MT_021
Processing MT_022
Processing MT_023
Processing MT_024
Processing MT_025
Processing MT_026
Processing MT_027
Processing MT_028
Processing MT_029
Processing MT_030
Processing MT_031
Processing MT_032
Processing MT_033
Processing MT_0

In [ ]:
# !chmod 777 run.sh
# !./run.sh
# !rm -r /content/drive/Shareddrives/CS726-Advanced-Machine-Learning/tft_tf2-master/tft_outputs/saved_models/traffic/fixed

In [ ]:
# !python3 script_train_fixed_params.py "traffic" "./tft_outputs" "no" "yes"

In [4]:
from script_train_fixed_params import main
import os

import data_formatters.electricity
import data_formatters.favorita
import data_formatters.traffic
import data_formatters.volatility

import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)


expt_name = "electricity"
use_gpu = True
model_folder = "/content/long_range_TSF/tft_outputs/saved_models/electricity/fixed"
data_csv_path = "/content/long_range_TSF/tft_outputs/data/electricity/hourly_electricity.csv"
data_formatter = data_formatters.electricity.ElectricityFormatter()
use_testing_mode = False

num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()
print("reached here")
# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

# Parameter overrides for testing only! Small sizes used to speed up script.
train_samples, valid_samples = 20000, 2000
if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 6
    train_samples, valid_samples = 100, 10
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                    fixed_params, model_folder)

# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
print(fixed_params)

for k in params:
    print("{}: {}".format(k, params[k]))

print(train_samples)

best_loss = np.Inf
for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

        tf.keras.backend.set_session(sess)

        params = opt_manager.get_next_parameters()
        model = ModelClass(params, use_cudnn=use_gpu)
        if not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)

        sess.run(tf.global_variables_initializer())
        model.fit()

        val_loss = model.evaluate()

        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss

        tf.keras.backend.set_session(default_keras_session)

Selecting GPU ID=0
*** Training from defined parameters for electricity ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
reached here
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
{'total_time_steps': 192, 'num_encoder_steps': 168, 'num_epochs': 100, 'early_stopping_patience': 5, 'multiprocessing_workers': 5, 'column_definition': [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)], 'input_size': 5, 'output_size': 1, 'category_counts': [369], 'input_obs_loc': [0]

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


20000/20000 [==============================] - 52s 3ms/sample - loss: 0.3701 - val_loss: 0.2693
Epoch 2/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2585 - val_loss: 0.2750
Epoch 3/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2426 - val_loss: 0.2562
Epoch 4/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2310 - val_loss: 0.2418
Epoch 5/100
20000/20000 [==============================] - 42s 2ms/sample - loss: 0.2258 - val_loss: 0.2639
Epoch 6/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2197 - val_loss: 0.2376
Epoch 7/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2162 - val_loss: 0.2312
Epoch 8/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2110 - val_loss: 0.2288
Epoch 9/100
20000/20000 [==============================] - 41s 2ms/sample - loss: 0.2088 - val_loss: 0.2210
Epoch 10/100
20000/20000 [==============

In [5]:
print("*** Running tests ***")
# import data_formatters.electricity
# import data_formatters.favorita
# import data_formatters.traffic
# import data_formatters.volatility

# import argparse
# import datetime as dte
# import os

# import data_formatters.base

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)
    print("Validation loss is : ", val_loss)
    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    print(type(output_map))
    print(output_map.keys())
    targets = data_formatter.format_predictions(output_map["targets"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])
print()
print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
    p50_loss.mean(), p90_loss.mean()))

Streaming output truncated to the last 5000 lines.
 [ 0.10360005]
 [-0.02356851]
 [-0.18932109]
 [-0.11114425]
 [-0.180955  ]
 [ 0.13738433]
 [ 0.04456463]
 [-0.11101685]
 [-0.16660999]
 [-0.11091039]
 [-0.02685559]
 [ 0.07783577]
 [-0.08739956]
 [-0.10187739]
 [-0.00448449]
 [-0.04847044]
 [-0.10252276]]
tensor: TemporalFusionTransformer/dense_5/bias (float32) [160]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
tensor: TemporalFusionTransformer/dense_5/kernel (float32) [160, 160]
[[ 0.00254805  0.13067439  0.03008613 ... -0.00349243 -0.11457326
   0.0

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Validation loss is :  0.23930846026641323
Computing test loss


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


<class 'dict'>
dict_keys(['p10', 'p50', 'p90', 'targets'])
Training completed @ 2022-05-05 21:50:32.166095
Best validation loss = 0.23930846026641323
Params:
dropout_rate  =  0.1
hidden_layer_size  =  160
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  /content/long_range_TSF/tft_outputs/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  100
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEG